In [1]:
import numpy as np
import pandas as pd
from scipy.linalg import solve
from scipy.stats import norm

S0, K, T, r, sigma = 50, 50, 1, 0.08, 0.3
Smax = 3 * K  # numerical infinity
M = 200       # space steps
N = 1000      # time steps in τ-space

# Transformation constants
qs = (r - 0.0) / (0.5 * sigma**2)  # non dividend paying
x_max = np.log(Smax / K)
x_min = -x_max
dx = (x_max - x_min) / M
dtau = (0.5 * sigma**2 * T) / N  # tau runs from 0 to sigma*sigma*T/2

# Grid
x_vals = np.linspace(x_min, x_max, M+1)
S_vals = K * np.exp(x_vals)  # Map back to S
idx_S0 = np.searchsorted(S_vals, S0)
# Initial condition for y(x, 0)
y0_call = np.maximum(np.exp(0.5 * (qs + 1) * x_vals) -
                     np.exp(0.5 * (qs - 1) * x_vals), 0)
y0_put  = np.maximum(np.exp(0.5 * (qs - 1) * x_vals) -
                     np.exp(0.5 * (qs + 1) * x_vals), 0)


lambda_const = dtau / dx**2


# Call option

def get_tau(t):
    return ((1-t)*0.09)/2

def r1_call(x, tau):  # x tends - infinity
    return 0.0

def r2_call(x, tau):  #  x tends plus infinity
    return np.exp(0.5 * (qs + 1) * x + 0.25 * (qs + 1)**2 * tau)

# Put option
def r1_put(x, tau):   #  x tends - infinity
    return np.exp(0.5 * (qs - 1) * x + 0.25 * (qs - 1)**2 * tau)

def r2_put(x, tau):   #  x tends plus infinity
    return 0.0


def boundary_vector(tau, tau_next, opt_type="call"):
    d = np.zeros(M-1)
    if opt_type == "call":
        d[0]  = (lambda_const / 2) * (r1_call(x_min, tau_next) + r1_call(x_min, tau))
        d[-1] = (lambda_const / 2) * (r2_call(x_max, tau_next) + r2_call(x_max, tau))
    elif opt_type == "put":
        d[0]  = (lambda_const / 2) * (r1_put(x_min, tau_next) + r1_put(x_min, tau))
        d[-1] = (lambda_const / 2) * (r2_put(x_max, tau_next) + r2_put(x_max, tau))
    return d


def tridiag(a, b, c, m):
    return np.diag([b]*m) + np.diag([a]*(m-1), -1) + np.diag([c]*(m-1), 1)

call_explicit , call_implicit , call_cn = [] , [] , []
put_explicit , put_implicit , put_cn = [] , [] , []

t_vals=[ 0,0.25,0.50,0.75,0.95]
t_vals.reverse()
tau_vals=[get_tau(t) for t in t_vals]
print("Values of taus corresponding to given ts")
print(tau_vals)
print()
time_steps=[(int)(t//dtau) for t in tau_vals]
print("Values of timesteps corresponding to given taus")
print(time_steps)
print()



def explicit_fd(y0, opt_type="call"):
    y = y0.copy()
    A = tridiag(lambda_const, 1 - 2*lambda_const, lambda_const, M-1)
    tau = 0.0
    for h in range(N):
        tau_next = tau + dtau
        d = boundary_vector(tau, tau_next, opt_type)
        y[1:M] = A @ y[1:M] + d
        if (h + 1) in time_steps:
            y_temp=y.copy()
            u=map_back(y_temp)
            if opt_type == "call":
                call_explicit.append(u[idx_S0])
            else:
                put_explicit.append(u[idx_S0])
        tau = tau_next
    return y


def implicit_fd(y0, opt_type="call"):
    y = y0.copy()
    A = tridiag(-lambda_const, 1 + 2*lambda_const, -lambda_const, M-1)
    tau = 0.0
    for h in range(N):
        tau_next = tau + dtau
        d = boundary_vector(tau, tau_next, opt_type)
        y[1:M] = solve(A, y[1:M] + d)
        if (h + 1) in time_steps:
            y_temp=y.copy()
            u=map_back(y_temp)
            if opt_type == "call":
                call_implicit.append(u[idx_S0])
            else:
                put_implicit.append(u[idx_S0])
        tau = tau_next
    return y


def crank_nicolson_fd(y0, opt_type="call"):
    y = y0.copy()
    AL = tridiag(-0.5*lambda_const, 1 + lambda_const, -0.5*lambda_const, M-1)
    AR = tridiag(0.5*lambda_const, 1 - lambda_const, 0.5*lambda_const, M-1)
    tau = 0.0
    for h in range(N):
        tau_next = tau + dtau
        d = boundary_vector(tau, tau_next, opt_type)
        y[1:M] = solve(AL, AR @ y[1:M] + d)
        if (h + 1) in time_steps:
            y_temp=y.copy()
            u=map_back(y_temp)
            if opt_type == "call":
                call_cn.append(u[idx_S0])
            else:
                put_cn.append(u[idx_S0])
        tau = tau_next
    return y


def map_back(y):
    return K * np.exp(-0.5*(qs - 1)*x_vals - r*T) * y


def bs_price(S, K, T, r, sigma, option_type='call'):
    d1 = (np.log(S/K) + (r + 0.5*sigma**2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma*np.sqrt(T)
    if option_type == 'call':
        return S*norm.cdf(d1) - K*np.exp(-r*T)*norm.cdf(d2)
    else:
        return K*np.exp(-r*T)*norm.cdf(-d2) - S*norm.cdf(-d1)
    
y_exp_call = explicit_fd(y0_call, opt_type="call")
y_imp_call = implicit_fd(y0_call, opt_type="call")
y_cn_call  = crank_nicolson_fd(y0_call, opt_type="call")

y_exp_put = explicit_fd(y0_put, opt_type="put")
y_imp_put = implicit_fd(y0_put, opt_type="put")
y_cn_put  = crank_nicolson_fd(y0_put, opt_type="put")

df = pd.DataFrame({'time' : t_vals 
                   , 'Explicit Call' : call_explicit , 'Implicit Call' : call_implicit , 'Crank-Nicolson call' : call_cn
                    , 'Explicit put' : put_explicit , 'Implicit put' : put_implicit , 'Crank-Nicolson put' : put_cn
                  })
print("Bs price for call at t=0,",bs_price(S0, K, T, r, sigma, option_type='call'))
print()
print("Bs price for put at t=0,",bs_price(S0, K, T, r, sigma, option_type='put'))
print()
print(df)



Values of taus corresponding to given ts
[0.002250000000000002, 0.01125, 0.0225, 0.03375, 0.045]

Values of timesteps corresponding to given taus
[50, 250, 500, 750, 1000]

Bs price for call at t=0, 7.855656273946487

Bs price for put at t=0, 4.011473593278275

   time  Explicit Call  Implicit Call  Crank-Nicolson call  Explicit put  \
0  0.95       1.331545       1.325388             1.328479      1.146494   
1  0.75       3.283261       3.280708             3.281985      2.349288   
2  0.50       5.007006       5.005398             5.006202      3.116968   
3  0.25       6.509252       6.508272             6.508762      3.639212   
4  0.00       7.926473       7.926322             7.926397      4.041090   

   Implicit put  Crank-Nicolson put  
0      1.140319            1.143419  
1      2.346647            2.347969  
2      3.115175            3.116072  
3      3.637822            3.638517  
4      4.040002            4.040546  
